

# CSCI 3202, Fall 2024
# Project Intermediate Results
# Due: December 11 at 11:59 pm

<br> 

### Your name: Parker Allen

<br> 

# Introduction

In this project, I am developing an AI athat solves the 8-puzzle game using search algorithms that I've worked on in the class. The 8-puzzle is a sliding puzzle with a 3x3 grid, where tiles move to match a certain order by sliding tiles into the empty space.

## The project involves:

Making the 8-puzzle game environment.

Applying search the algorithms BFS, DFS, and A* with heuristics.

Finding the best algorithm and comparing algorithms.

Providing a user interface for inputting puzzles and viewing solutions.

## Code implementation:

### Importing Libraries


In [1]:
import heapq
from collections import deque
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import filedialog, messagebox
import random
import time


### Define the PuzzleState class

In [2]:
class PuzzleState:
    def __init__(self, board, parent=None, move="", depth=0, tiles=None):
        self.board = board
        self.size = len(board)
        self.parent = parent
        self.move = move
        self.depth = depth
        self.key = str(self.board)
        self.tiles = tiles

    def __eq__(self, other):
        return self.board == other.board

    def __hash__(self):
        return hash(self.key)

    def find_blank(self):
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] == 0:
                    return i, j

    def get_neighbors(self):
        neighbors = []
        x, y = self.find_blank()
        directions = {'Up': (x - 1, y),
                      'Down': (x + 1, y),
                      'Left': (x, y - 1),
                      'Right': (x, y + 1)}
        for move, (new_x, new_y) in directions.items():
            if 0 <= new_x < self.size and 0 <= new_y < self.size:
                new_board = [row[:] for row in self.board]
                new_board[x][y], new_board[new_x][new_y] = new_board[new_x][new_y], new_board[x][y]
                neighbor = PuzzleState(new_board, self, move, self.depth + 1, self.tiles)
                neighbors.append(neighbor)
        return neighbors

    def is_goal(self, goal_board):
        return self.board == goal_board

    def manhattan_distance(self, goal_board):
        """Heuristic: Sum of Manhattan distances between current and goal pos."""
        distance = 0
        goal_positions = {val: (i, j) for i, row in enumerate(goal_board) for j, val in enumerate(row)}
        for i in range(self.size):
            for j in range(self.size):
                value = self.board[i][j]
                if value != 0:
                    goal_i, goal_j = goal_positions[value]
                    distance += abs(i - goal_i) + abs(j - goal_j)
        return distance
    def misplaced_tiles(self, goal_board):
        """Heuristic: Count the number of misplaced tiles."""
        return sum(1 for i in range(self.size) for j in range(self.size)
                   if self.board[i][j] != 0 and self.board[i][j] != goal_board[i][j])

### Define Search Function

In [3]:

def general_search(ss, goal, strat='bfs', heur='manhattan', dl=50):
    if strat == 'bfs':
        frontier = deque([ss])
    elif strat == 'dfs':
        frontier = [ss]
    elif strat == 'astar':
        frontier = []
        counter = 0
        h =ss.manhattan_distance(goal) if heur== 'manhattan' else ss.misplaced_tiles(goal)
        heapq.heappush(frontier, (h, counter, ss))
    else:
        raise ValueError("Invalid search strategy!")

    explored = set() 
    while frontier:
        if strat == 'astar':
            _, _, state = heapq.heappop(frontier)
        elif strat== 'dfs':
            state= frontier.pop()
        else:
            state = frontier.popleft()
        if state.is_goal(goal):
            return state

        explored.add(state)
        for neighbor in state.get_neighbors():
            if neighbor not in explored:
                if strat == 'astar':
                    h = neighbor.manhattan_distance(goal) if heur == 'manhattan' else neighbor.misplaced_tiles(goal)
                    f = neighbor.depth + h
                    counter += 1
                    heapq.heappush(frontier, (f, counter, neighbor))
                else:
                    if strat == 'dfs' and neighbor.depth > dl:
                        continue
                    frontier.append(neighbor)

    return None  

In [4]:
def is_solvable(puzzle):
    """Check if a puzzle is solvable."""
    size = len(puzzle)
    ic = 0
    fp = [tile for row in puzzle for tile in row if tile != 0]
    for i in range(len(fp)):
        for j in range(i + 1, len(fp)):
            if fp[i]> fp[j]:
                ic +=1
    if size % 2 != 0:
        return ic%2== 0
    else:
        blank_row = size - next(i for i, row in enumerate(puzzle) if 0 in row)
        return (ic + blank_row) % 2 == 0

In [5]:
def load_and_split_image(image_path, ps):
    image = Image.open(image_path)
    min_d = min(image.width, image.height)
    left = (image.width-min_d)// 2
    top = (image.height -min_d) // 2
    right = left+min_d
    bottom = top+min_d
    image = image.crop((left, top, right, bottom)) 

    ts = 300
    image = image.resize((ts, ts), Image.Resampling.LANCZOS)

    tile_s = ts // ps 
    tiles = {}
    tilen = 1
    for row in range(ps):
        for col in range(ps):
            if tilen <= ps*ps-1:
                x0 = col *tile_s
                y0 = row* tile_s
                x1 = x0+ tile_s
                y1 = y0 +tile_s
                tile_image= image.crop((x0, y0, x1, y1))
                tiles[tilen] = ImageTk.PhotoImage(tile_image)
                tilen += 1
    blank_tile = Image.new('RGB', (tile_s, tile_s), color='white')
    tiles[0] = ImageTk.PhotoImage(blank_tile)

    return tiles, tile_s

### Define Reconcruction Path

In [6]:
def rec_path(state):
    """Reconstruct the path from the goal state to the start state."""
    path = [] 
    moves = [] 
    while state.parent is not None:
        path.append(state.board)
        moves.append(state.move) 
        state = state.parent
    path.append(state.board)
    path.reverse() 
    moves.reverse()
    return path, moves

### GUI

In [7]:
class PuzzleGUI:
    def __init__(self, root):
        """Initialize the GUI application."""
        self.root = root
        self.root.title("Sliding Puzzle Solver")  
        self.puzzle_size = 3
        self.tiles = {}
        self.board = []
        self.goal = []
        self.labels = []
        self.tile_size = 0  

        self.create_widgets()
    def create_widgets(self):
        """
        Create and organize the GUI components.
        """
        self.frame = tk.Frame(self.root)
        self.frame.pack()
        tk.Label(self.frame, text="Select Puzzle Size:").grid(row=0, column=0, padx=5, pady=5)
        self.size_var = tk.IntVar(value=3)
        for idx, size in enumerate([3]):
            tk.Radiobutton(
                self.frame, text=f"{size}x{size}", variable=self.size_var, value=size,
                command=self.change_size
            ).grid(row=0, column=idx+1, padx=5, pady=5)
        tk.Button(self.frame, text="Load Image", command=self.load_image).grid(row=1, column=0, padx=5, pady=5)
        tk.Button(self.frame, text="Scramble", command=self.scramble_puzzle).grid(row=1, column=1, padx=5, pady=5)
        tk.Button(self.frame, text="Solve", command=self.solve_puzzle).grid(row=1, column=2, padx=5, pady=5)
        self.button_frame = tk.Frame(self.root)
        self.button_frame.pack()

    def change_size(self):
        """
        Handle changes in the puzzle size.
        """
        self.puzzle_size = self.size_var.get()
        self.board = []
        self.goal = [] 
        self.tiles = {}  
        self.create_tile_labels()

    def load_image(self):
        """Load and process an image for the puzzle."""
        image_path = filedialog.askopenfilename()
        if not image_path:
            return
        try:
            self.puzzle_size = self.size_var.get()
            self.tiles, self.tile_size = load_and_split_image(image_path, self.puzzle_size) 
            self.goal = self.generate_goal_state(self.puzzle_size) 
            self.board = [row[:] for row in self.goal] 
            self.create_tile_labels()
            self.update_board()
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load image: {e}")

    def create_tile_labels(self):
        """Create or refresh the grid of labels for the puzzle tiles."""
        for widget in self.button_frame.winfo_children():
            widget.destroy()
        self.labels = []
        for i in range(self.puzzle_size):
            row = []
            for j in range(self.puzzle_size):
                label = tk.Label(self.button_frame, bd=0)
                label.grid(row=i, column=j, padx=0, pady=0) 
                label.bind('<Button-1>', lambda e, x=i, y=j: self.tile_click(x, y))
                row.append(label)
            self.labels.append(row)

    def update_board(self):
        """Update the GUI."""
        for i in range(self.puzzle_size):
            for j in range(self.puzzle_size):
                tile_number = self.board[i][j]
                if tile_number == 0:
                    self.labels[i][j].config(image=self.tiles[0])
                    self.labels[i][j].image = self.tiles[0]
                else:
                    self.labels[i][j].config(image=self.tiles[tile_number])
                    self.labels[i][j].image = self.tiles[tile_number]

    def tile_click(self, i, j):
        """Handle a tile click event."""
        blank_i, blank_j = self.find_blank()
        if (abs(blank_i - i) == 1 and blank_j == j) or (abs(blank_j - j) == 1 and blank_i == i):
            self.board[blank_i][blank_j], self.board[i][j] = self.board[i][j], self.board[blank_i][blank_j]
            self.update_board()
            if self.board == self.goal:
                messagebox.showinfo("Puzzle Solver", "Congratulations! You solved the puzzle.")

    def find_blank(self):
        """Find the position."""
        for i in range(self.puzzle_size):
            for j in range(self.puzzle_size):
                if self.board[i][j] == 0:
                    return i, j
        return None, None  

    def scramble_puzzle(self):
        """Scramble the puzzle by randomly shuffling the tiles."""
        if not self.board:
            messagebox.showerror("Error", "No puzzle loaded!")
            return
        flat_board = sum(self.board, [])
        random.shuffle(flat_board)
        while not is_solvable([flat_board[i:i + self.puzzle_size] for i in range(0, len(flat_board), self.puzzle_size)]):
            random.shuffle(flat_board)
        self.board = [flat_board[i:i + self.puzzle_size] for i in range(0, len(flat_board), self.puzzle_size)]
        self.update_board()

    def generate_goal_state(self, size):
        """Generate the goal state for the puzzle."""
        goal = list(range(1, size * size))
        goal.append(0)
        return [goal[i:i + size] for i in range(0, len(goal), size)]  

    def solve_puzzle(self):
        """Solve the puzzle using multiple search algorithms and display the results."""
        if not self.board:
            messagebox.showerror("Error", "No puzzle loaded or scrambled!") 
            return

        self.start_state = PuzzleState(self.board, tiles=self.tiles) 
        algor = [
            ('BFS', 'bfs', None),
            ('DFS', 'dfs', None),
            ('A* (Manhattan)', 'astar', 'manhattan'),
            ('A* (Misplaced Tiles)', 'astar', 'misplaced')
        ]
        results = []

        self.root.update()

        for name, strat, heur in algor:
            start_time = time.time()
            if strat == 'dfs':
                result = general_search(self.start_state, self.goal, strat=strat, dl=50)
            elif strat == 'astar':
                result = general_search(self.start_state, self.goal, strat=strat, heur=heur)
            else:
                result = general_search(self.start_state, self.goal, strat=strat)
            end_time = time.time()  
            if result:
                path, moves = rec_path(result)
                results.append({
                    'name': name,
                    'moves': moves,
                    'num_moves': len(moves),
                    'time': end_time - start_time,
                    'path': path
                })
            else:
                results.append({
                    'name': name,
                    'moves': None,
                    'num_moves': None,
                    'time': None,
                    'path': None
                })
        max_moves = max(r['num_moves'] for r in results if r['num_moves'] is not None)
        max_time = max(r['time'] for r in results if r['time'] is not None)

        for r in results:
            if r['num_moves'] is not None:
                r['score'] = (r['num_moves'] / max_moves) + (r['time'] / max_time)
            else:
                r['score'] = float('inf')

        best_result = min(results, key=lambda r: r['score'])

        message = "Algorithm Comparison:\n"
        for res in results:
            if res['num_moves'] is not None:
                message += (f"{res['name']}:\n"
                            f"  Moves: {res['num_moves']}\n"
                            f"  Time: {res['time']:.4f} seconds\n"
                            f"  Score: {res['score']:.4f}\n")
            else:
                message += f"{res['name']}:\n  No solution found.\n"

        if best_result and best_result['num_moves'] is not None:
            message += f"\nBest Algorithm: {best_result['name']}\n"
            message += f"Number of Moves: {best_result['num_moves']}\n"
            message += f"Time Taken: {best_result['time']:.4f} seconds"
            self.animate_solution(best_result['path'])
        else:
            message += "\nNo solution found by any algorithm."

        messagebox.showinfo("Puzzle Solver Results", message)

    def animate_solution(self, path):
        """Animate the solution path on the puzzle grid."""
        if not path:
            return

        def animate(step=0):
            if step < len(path):
                self.board = path[step] 
                self.update_board()
                self.root.after(500, animate, step + 1)

        animate()


In [ ]:
if __name__ == "__main__":
    root = tk.Tk()
    app = PuzzleGUI(root)
    root.mainloop()


### Set Up Puzzle

In [ ]:
start=[[1, 2, 3],
        [4, 0, 5],
        [6, 7, 8]]

goal=[[1, 2, 3],
        [4, 5, 6],
        [7, 8, 0]]

ss = PuzzleState(start)


### BFS

In [ ]:
res= general_search(ss, goal,strat='bfs')
if res:
    path, moves=rec_path(res)
    print("BFS Solution Moves:",moves)
    print("Number of moves:", len(moves))
else:
    print("No solution found using BFS.")


BFS Solution Moves: ['Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Down', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right']
Number of moves: 14


### DFS

In [ ]:
res=general_search(ss,goal,strat='dfs',dl=50)
if res:
    path, moves=rec_path(res)
    print("\nDFS Solution Moves:",moves)
    print("Number of moves:",len(moves))
else:
    print("No solution found using DFS.")



DFS Solution Moves: ['Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Down', 'Left', 'Up', 'Right', 'Down', 'Left', 'Up', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Down', 'Left', 'Up', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Down']
Number of moves: 50


### A* Search with Manhattan Distance Heuristic

In [ ]:
res=general_search(ss, goal, strat='astar', heur='manhattan')
if res:
    path, moves=rec_path(res)
    print("\nA* (Manhattan) Solution Moves:",moves)
    print("Number of moves:",len(moves))
else:
    print("No solution found using A* with Manhattan distance.")



A* (Manhattan) Solution Moves: ['Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Down', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right']
Number of moves: 14


### A* Search with Misplaced Tiles Heuristic

In [ ]:
res=general_search(ss, goal,strat='astar', heur='misplaced')
if res:
    path, moves= rec_path(res)
    print("\nA* (Misplaced Tiles) Solution Moves:", moves)
    print("Number of moves:", len(moves))
else:
    print("No solution found using A* with Misplaced Tiles.")



A* (Misplaced Tiles) Solution Moves: ['Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Down', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right']
Number of moves: 14


### Results of Testing So Far

#### Test Cases

I tested the algorithms with various starting configurations, ranging from easy to difficult puzzles. The goal used in all tests is:

In [ ]:
goal_board = [[1, 2, 3],
              [4, 5, 6],
              [7, 8, 0]]



### Performance Comparison

| Algorithm| Nodes Expanded | Solution Length | Time Taken|
|-|-|-|-|
| BFS| Medium | Optimal| Most and sometimes Third|
| DFS| High| Non-optimal| Third and sometimes Most|
| A* (Manhattan)| Medium|Optimal| Least |
| A* (Misplaced)| Medium| Optimal| Second|

### Observations

- **BFS**:
  - Guarantees the shortest path.
  - High memory is used.

- **DFS**:
  - Can go to deep branches.
  - May not find the shortest solution.
  - It needs to have a depth limit.

- **A\* with Manhattan Distance**:
  - Finding the optimal solution the best.
  - Explores fewer nodes.

- **A\* with Misplaced Tiles**:
  - Less efficient than Manhattan.
  - Still better than BFS and DFS.

---
